In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import logit
from scipy.stats import gaussian_kde, qmc
from scipy.integrate import nquad

In [2]:
# 载入数据
data = pd.read_csv(r'D:\my work\Causal theory\pisa数据\2022\CY08MSP_STU_QQQ_processed_Cal-CounterFactual.csv')

# 建立逻辑回归模型
model = logit("ICTUSE ~ ICTSCHPRD + ICTQUAL + ICTREG", data=data).fit()

# 显示模型结果
print(model.summary())

# 提取 ICTQUAL 和 ICTREG 的数据
W_values = data[['ICTQUAL', 'ICTREG']].values.T

# 建立核密度估计对象
kde = gaussian_kde(W_values)

def estimate_conditional_probability(x_value, num_trials=3, num_samples=4096): # 调整num_sample以平衡精度和性能
    probabilities = []
    for trial in range(num_trials):
        # 创建一个准随机数生成器
        sampler = qmc.Sobol(d=2, scramble=True)
        
        # 生成样本
        sample_w1_w2 = qmc.scale(sampler.random(n=num_samples), [data['ICTQUAL'].min(), data['ICTREG'].min()], [data['ICTQUAL'].max(), data['ICTREG'].max()])
        
        total_probability = 0
        total_weight = 0
        for w1, w2 in sample_w1_w2:
            density = kde.evaluate([w1, w2])[0]
            # 估计 Y = 1 的概率给定 X=x, W1=w1, W2=w2
            p_y_given_x_w1_w2 = model.predict(exog=dict(ICTSCHPRD=x_value, ICTQUAL=w1, ICTREG=w2))
            total_probability += p_y_given_x_w1_w2 * density
            total_weight += density
        probability = total_probability / total_weight
        print(f"第{trial+1}次实验的结果为{probability}")
        probabilities.append(probability)
    return probabilities, np.mean(probabilities)  # 返回多次实验的平均概率

# 计算并输出结果
probabilities_x1, p_y_x1 = estimate_conditional_probability(1)
probabilities_x0, p_y_x0 = estimate_conditional_probability(0)

print(f"P(ICTUSE=1 | do(ICTSCHPRD=1)): {p_y_x1}")
print(f"P(ICTUSE=1 | do(ICTSCHPRD=0)): {p_y_x0}")

Optimization terminated successfully.
         Current function value: 0.492867
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 ICTUSE   No. Observations:               256492
Model:                          Logit   Df Residuals:                   256488
Method:                           MLE   Df Model:                            3
Date:                Wed, 19 Jun 2024   Pseudo R-squ.:                 0.02408
Time:                        17:49:42   Log-Likelihood:            -1.2642e+05
converged:                       True   LL-Null:                   -1.2954e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1510      0.033     -4.566      0.000      -0.216      -0.086
ICTSCHPRD      1.6103      0.

In [3]:
probabilities_x1, probabilities_x0

([0    0.801653
  dtype: float64,
  0    0.801659
  dtype: float64,
  0    0.801654
  dtype: float64],
 [0    0.45391
  dtype: float64,
  0    0.453945
  dtype: float64,
  0    0.453921
  dtype: float64])

In [4]:

# 计算 P(ICTUSE=1 | ICTSCHPRD=1)
ict_schprd_1 = data[data['ICTSCHPRD'] == 1]
p_ictuse_given_ictschprd_1 = ict_schprd_1[ict_schprd_1['ICTUSE'] == 1].shape[0] / ict_schprd_1.shape[0]

# 计算 P(ICTUSE=1 | ICTSCHPRD=0)
ict_schprd_0 = data[data['ICTSCHPRD'] == 0]
p_ictuse_given_ictschprd_0 = ict_schprd_0[ict_schprd_0['ICTUSE'] == 1].shape[0] / ict_schprd_0.shape[0]

p_ictuse_given_ictschprd_1, p_ictuse_given_ictschprd_0


(0.8017842506028676, 0.46746011648518615)

In [5]:
data

,ICTSCHPRD,ICTQUAL,ICTREG,ICTUSE
0,1,-0.3516,2.8904,1
1,1,1.0161,2.0196,1
2,1,1.1978,2.8904,1
3,1,-1.0266,0.8842,1
4,1,-0.1076,0.4704,0
...,...,...,...,...
256487,1,0.3623,1.0594,1
256488,1,0.2524,-0.5927,1
256489,1,0.3623,1.0594,0
256490,1,-2.8028,0.5826,0
